In [ ]:
%load_ext autoreload
%autoreload 2

Declaration of parameters (you must also add a tag for this cell - parameters)

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
    "type_export" : "torch_export",   # ["mmdeploy_export", "torch_export"]
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ 
      { STEP_NAME: "2_model_train", ENTITY_NAME: "model"}
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "cache_data" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

![interface 0_configure_pack_interface.drawio](./imgs/0_configure_pack_interface.drawio.png)

In [ ]:
#4 get substep.interface
inputs = substep.inputs(step_name = "2_model_train")
tmp_outputs = substep.tmp_outputs()

print(f"{inputs.model=}")

print(f"{tmp_outputs.cache_data=}")

In [ ]:
# Create dir by URLs of tmp_outputs
import os
import os.path as osp
os.makedirs(tmp_outputs.cache_data, exist_ok=True)

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

#### Loading a trained model from the model_train component 
(weights, configs)

In [ ]:
from sinara.store import SinaraStore
from pathlib import Path

# copy config from previos step to outputs
SinaraStore.copy_store_files_to_tmp(store_dir=inputs.model, tmp_dir=tmp_outputs.cache_data)
# Create SUCCESS file after copy from store
Path(osp.join(tmp_outputs.cache_data, '_SUCCESS')).touch()

In [ ]:
!ls -lh {tmp_outputs.cache_data}

#### Reading the configuration file and adding parameters

In [ ]:
import json
config_fn = os.path.join(tmp_outputs.cache_data, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)

In [ ]:
CONFIG

In [ ]:
DEVICE = "cuda:0"
MODEL_NAME = CONFIG["train_config_parameters"]["MODEL_NAME"]
CONFIG['work_dir'] = tmp_outputs.cache_data
WORK_DIR = CONFIG['work_dir']
ONNX_DIR = os.path.join(WORK_DIR, "onnx")
CONFIG['onnx_dir'] = ONNX_DIR
CONFIG['device'] = DEVICE

TYPE_EXPORT = substep_params["type_export"]
assert TYPE_EXPORT in ["mmdeploy_export", "torch_export"]

test_image_path = osp.join(tmp_outputs.cache_data, "test.jpg")
assert osp.exists(test_image_path)

Selecting a weights file to convert to onnx format (best, latest era, etc.)

In [ ]:
# Select model files
model_path = tmp_outputs.cache_data
save_best  = False
files = [osp.join(model_path, file) for file in os.listdir(model_path)]

models_pth  = [file for file in files if '.pth' in file and not os.path.islink(file)]
best_models = [file for file in models_pth if 'best' in file]
latest_models = [file for file in models_pth if 'latest' in file]

assert len(best_models), 'model have not pth files.'

if save_best and len(best_models) > 0:
    models_pth = best_models
else:
    models_pth = latest_models
    
models_pth.sort(key=lambda file: osp.getmtime(file))

last_model = models_pth[-1]
cfg_model = osp.join(CONFIG['work_dir'], "last_cfg.py")

last_model_onnx = f"{osp.splitext(osp.basename(last_model))[0]}.onnx"
print(f"{last_model=}")
print(f"{last_model_onnx=}")

In [ ]:
import mmdeploy
from mmcv import Config, ConfigDict
import mmcv
import mmdet
from mmdeploy.utils import load_config

print(f"{mmcv.__version__=}")
print(f"{mmdet.__version__=}")
print(f"{mmdeploy.__version__=}")


cfg = Config.fromfile(cfg_model)

cfg_deploy_dir = osp.join("/home/jovyan/mmdeploy", 'configs')
cfg_deploy_path = "mmdet/detection/detection_onnxruntime_dynamic.py"
cfg_deploy_path = osp.join(cfg_deploy_dir, cfg_deploy_path)
# cfg_deploy = Config.fromfile(cfg_deploy_path)

cfg_deploy, cfg = load_config(cfg_deploy_path, cfg_model)

cfg_deploy.onnx_config.save_file = f"{osp.splitext(osp.basename(last_model))[0]}.onnx"
print(cfg_deploy.pretty_text)

Writing model conversion configs to onnx format (`deploy.json`, `pipeline.json` and `detail.json`)

In [ ]:
os.makedirs(ONNX_DIR, exist_ok=True)

In [ ]:
CONFIG['config_model'] = cfg_model
CONFIG['config_deploy'] = cfg_deploy_path
CONFIG['torch_model'] = last_model
CONFIG['onnx_model'] = last_model_onnx
CONFIG['type_export'] = TYPE_EXPORT
CONFIG['test_image_path'] = test_image_path

In [ ]:
CONFIG['train_config_parameters']["Normalize"] = {'mean': [123.675, 116.28, 103.53],
                                                  'std': [58.395, 57.12, 57.375],
                                                  'to_rgb': True}

In [ ]:
with open(config_fn, 'w') as f_id:
    json.dump(CONFIG, f_id, indent=4)

In [ ]:
# Create SUCCESS file after successful copy
Path(osp.join(tmp_outputs.cache_data, '_SUCCESS')).touch()

In [ ]:
#stop spark
SinaraSpark.stop_session()